# Process power curve from external source (thewindpower.net)
This script illustrates how to process power curves from an external source (thewindpower.net) to use them within RESKit.



In [3]:
import reskit as rk
import pandas as pd
import numpy as np
import shutil
import os
# get base path of current file
base_path = os.path.dirname(__file__)

In [9]:
# Define path to the power curves and turbine library from thewindpower.net
path_power_curves = r"Power_curves_20230708.xls"
path_turbine_library = r"Turbines_20230708.xls"

# Note: specify the path to the RESKit repo
PATH_TO_RESKIT_REPO = os.path.join(base_path, "..")
output_folder = os.path.join("PATH_TO_RESKIT_REPO", "reskit", "wind", "core", "data", "turbines")
# specify the source of the data for information purposes
source = "World Wind Farms 2023"

In [13]:
df_power_curves = pd.read_excel(path_power_curves, sheet_name="Power_curves",header=[0,1])

df_turbine_library = pd.read_excel(path_turbine_library, sheet_name="Turbines", skiprows=[1])
df_turbine_library["Name"] = df_turbine_library["Name"].astype(str)
df_turbine_library["Manufacturer"] = df_turbine_library["Manufucturer"].astype(str)  # also rename typo

template_dict = {
    "Manufacturer": None,
    "Model": None,
    "Capacity": None,
    "Usage": None,
    "HubHeight": None,
    "Source": source,
    "RotorDiameter": None,
    "Power curve": None,
    "windspeed(m/s)": "power(kW)"
}


ws_list = df_power_curves.columns.levels[1].tolist()[0:71]


In [14]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i, row in df_power_curves.iterrows():
    
    single_turbine_df = pd.DataFrame.from_dict(template_dict, orient="index").squeeze()
    
    # turbine_lib
    turbine_id = row["Turb. ID"].values[0]
    turbine_lib_info = df_turbine_library.loc[df_turbine_library["ID"] == turbine_id].squeeze()
    
    single_turbine_df.loc["Manufacturer"] = row["Manufucturer Name"].values[0]
    manufacturer = row["Manufucturer Name"].values[0].replace(" ", "_")
    model_TWP = row["Turbine Name"].astype(str).values[0]
    model_RESKit = model_TWP.replace("/", "-").replace(" ", "_")
    
    # manufacturer is added to avoid duplicates
    model_RESKit = f"{model_RESKit}_{manufacturer}"
    
    single_turbine_df.loc["Model"] = model_RESKit
    single_turbine_df.loc["Capacity"] = turbine_lib_info["Rated power"]
    if turbine_lib_info["Offshore"] == "No":
        onshore_offshore = "Onshore"
    elif turbine_lib_info["Offshore"] == "Yes":
        onshore_offshore = "Offshore"
    else:
        onshore_offshore = None
    
    single_turbine_df.loc["Usage"] = onshore_offshore
    min_hub_height = turbine_lib_info["Minimum hub height"]
    max_hub_height = turbine_lib_info["Maximum hub height"]
    
    if min_hub_height == "#ND":
        min_hub_height = np.nan
        max_hub_height = np.nan
    
    
    single_turbine_df.loc["HubHeight"] = f"{min_hub_height}, {max_hub_height}"
    single_turbine_df.loc["RotorDiameter"] = float(turbine_lib_info["Rotor diameter"])
    
    # read power curve
    pc_values = row[4:75].values
    pc_df_turbine = pd.Series(pc_values, index=ws_list)   
    
    single_turbine_df = pd.concat([single_turbine_df, pc_df_turbine], axis=0)

    single_turbine_df.to_csv(f"{output_folder}/{model_RESKit}.csv", index=True, header=False)

Manufacturer          AAER
Model           A1000_AAER
Capacity              1000
Usage              Onshore
HubHeight           70, 82
                   ...    
33.0                     0
33.5                     0
34.0                     0
34.5                     0
35.0                     0
Length: 80, dtype: object